#### *Trabalho de Processamento de Big Data | Licenciatura em Ciência de Dados 2023/24 | CDB1*

Docente: João Oliveira<br><br>

- David Franco, nº110733

- Felipe Pereira, nº110861

- João Dias, nº110305

- Samuel Ricardo, nº110884<br><br>

https://www.kaggle.com/datasets/PROPPG-PPG/hourly-weather-surface-brazil-southeast-region

Version 9 (10.11 GB) | Created by John Holz | Data Update 2023/01/30

## Análise Exploratória de Dados

Esta fase corresponde à segunda fase do trabalho, neste notebook será realizado um conjunto de estatísticas descritivas das variáveis resultantes da fase anterior e que foram importadas através do ficheiro parquet. É importante realçar que, foram feitas apenas as estatísticas das variáveis consideradas mais importantes e que tiveram uma posterior utilização na fase da modelação. 

### Setup inicial

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import col, count, row_number, date_format, mean, min, max, stddev, year, month, avg, sum

from pyspark.sql.window import Window

In [2]:
# Some imports 

import os, sys

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Create the Spark session

spark = SparkSession.builder \
    .appName("WeatherBrazilEDA") \
    .config("spark.sql.shuffle.partitions", 512) \
    .config("spark.driver.memory", "128g") \
    .config("spark.executor.memory", "128g") \
    .getOrCreate()

In [4]:
spark

### Importação do ficheiro parquet


In [5]:
df_all_stations = spark.read.parquet("./all_stations.parquet")

In [6]:
print("Número de linhas: ",df_all_stations.count())
print("Número de colunas: ", len(df_all_stations.columns))

Número de linhas:  49759514
Número de colunas:  26


In [7]:
df_all_stations.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Hour: string (nullable = true)
 |-- TotalHourlyPrecipitationMm: double (nullable = true)
 |-- HourlyStationLevelAtmosphericPressureMb: double (nullable = true)
 |-- LastHourMaxAtmosphericPressureMb: double (nullable = true)
 |-- LastHourMinAtmosphericPressureMb: double (nullable = true)
 |-- HourlyDryBulbAirTemperatureC: double (nullable = true)
 |-- DewPointTemperatureC: double (nullable = true)
 |-- LastHourMaxTemperatureC: double (nullable = true)
 |-- LastHourMinTemperatureC: double (nullable = true)
 |-- LastHourMaxDewPointTemperatureC: double (nullable = true)
 |-- LastHourMinDewPointTemperatureC: double (nullable = true)
 |-- LastHourMaxRelativeHumidityPercentage: integer (nullable = true)
 |-- LastHourMinRelativeHumidityPercentage: integer (nullable = true)
 |-- HourlyRelativeHumidityPercentage: integer (nullable = true)
 |-- HourlyWindDirectionRadiusDegrees: integer (nullable = true)
 |-- Maximum

### Análise descritiva das variáveis numéricas


In [6]:
# Adicionar uma coluna que é a média das temperaturas máxima e mínima da última hora
df_all_stations = df_all_stations.withColumn(
    "AverageTemperatureC",
    (col("LastHourMaxTemperatureC") + col("LastHourMinTemperatureC")) / 2
)

In [7]:
# Colunas numéricas
numeric_columns = [
    'TotalHourlyPrecipitationMm', 'HourlyStationLevelAtmosphericPressureMb',
    'LastHourMaxAtmosphericPressureMb', 'LastHourMinAtmosphericPressureMb',
    'HourlyDryBulbAirTemperatureC', 'DewPointTemperatureC',
    'LastHourMaxTemperatureC', 'LastHourMinTemperatureC', "AverageTemperatureC",
    'LastHourMaxDewPointTemperatureC', 'LastHourMinDewPointTemperatureC',
    'LastHourMaxRelativeHumidityPercentage', 'LastHourMinRelativeHumidityPercentage',
    'HourlyRelativeHumidityPercentage', 'HourlyWindDirectionRadiusDegrees',
    'MaximumWindGustMs', 'HourlyWindSpeedMs','Height'
]


In [8]:
desc_stats = df_all_stations.select(numeric_columns).describe().toPandas().transpose()
desc_stats

,0,1,2,3,4
summary,count,mean,stddev,min,max
TotalHourlyPrecipitationMm,49759514,0.15276609011914452,1.2644366957327162,0.0,100.0
HourlyStationLevelAtmosphericPressureMb,49759514,965.2958074570444,37.44976884102553,720.1,1050.0
LastHourMaxAtmosphericPressureMb,49759514,965.5655641713201,37.45299070576381,720.3,1050.0
LastHourMinAtmosphericPressureMb,49759514,965.0045343790938,37.436673636584565,720.0,1049.6
HourlyDryBulbAirTemperatureC,49759514,23.41319106533029,5.549682278526645,-7.7,45.0
DewPointTemperatureC,49759514,17.177764288453435,4.760310674870106,-27.5,44.7
LastHourMaxTemperatureC,49759514,24.07602705886555,5.6797061254375425,-7.6,45.0
LastHourMinTemperatureC,49759514,22.783562140498123,5.4138710682566815,-9.5,44.9
AverageTemperatureC,49759514,23.42979459968197,5.5262808169749835,-7.699999999999999,44.95


Analisando a tabela acima é possível verificar que, todas as colunas contêm o mesmo número de observações, o que implica não existir valores omissos, isto é um comportamento esperado, uma vez que, os valores omissos foram tratados no notebook anterior. Em relação à média e desvio padrão das variáveis é bastante difícil de generalizar, uma vez que, os valores variam bastante tendo em conta a variável em conta, para além do facto, de cada variável ter uma unidade de medida própria.

### Análise descritiva de variáveis importantes


#### Número de Registos por Região

In [34]:
count_by_region = df_all_stations.groupBy("Region").agg(
    count("*").alias("Number of Records")
).orderBy(col("Number of Records").desc())

count_by_region.show()

+------+-----------------+
|Region|Number of Records|
+------+-----------------+
|    SE|         13274895|
|    NE|         12934432|
|    CO|          8888790|
|     S|          8796918|
|     N|          5864479|
+------+-----------------+



A tabela mostra grandes variações no número de registos registados nas diferentes regiões brasileiras, o que nos indica uma disparidade no número de estações entre cada região. A região sudeste é que a regista maior número de registos enquanto a norte é a que tem menos registos.

#### Número de Registos por Estação Meteorológica

In [35]:
count_by_station = df_all_stations.groupBy("Station").agg(
    count("*").alias("Number of Records")
).orderBy(col("Number of Records").desc())

count_by_station.show()

+--------------------+-----------------+
|             Station|Number of Records|
+--------------------+-----------------+
|             VALENCA|           189499|
|            BRASILIA|           178013|
|        PORTO ALEGRE|           172448|
|             GOIANIA|           158220|
|               ARAXA|           153005|
|           PETROLINA|           150445|
|       MONTES CLAROS|           149264|
|               BAURU|           148197|
|       FLORIANOPOLIS|           147221|
|         SANTA MARIA|           145477|
|          UBERLANDIA|           144192|
|            SALVADOR|           143794|
|           ITABERABA|           143565|
|             ITIRUCU|           143534|
|   ECOLOGIA AGRICOLA|           142879|
|               BELEM|           142239|
|              MANAUS|           141211|
|          PONTA PORA|           140105|
|LUIZ EDUARDO MAGA...|           139754|
|            ALMENARA|           138826|
+--------------------+-----------------+
only showing top

A tabela demonstra que a estação de Valença e que a regista o maior número de regisots seguida da estação de Brasília e Porto Alegre que têm valores relativamente próximos. Esta distribuição sugere uma grande variação no volume de dados registado por estação, que se pode explicar, talvez, pela antiguidade da implentação das estações.

#### Média da Temperatura por Ano

In [47]:
# Agrupar por ano e calcular a média da temperatura média
avg_temp_by_year = df_all_stations.groupBy(year("Date").alias("Year")).agg(
    avg("AverageTemperatureC").alias("Average Temperature")
).orderBy(col("Year").desc())

avg_temp_by_year.show(n=avg_temp_by_year.count(), truncate=False)

+----+-------------------+
|Year|Average Temperature|
+----+-------------------+
|2021|24.01397917069357  |
|2020|23.34789584957835  |
|2019|23.861435282877746 |
|2018|23.363212864566414 |
|2017|23.582905358743727 |
|2016|23.704758138980417 |
|2015|24.038375720816866 |
|2014|23.562245877936792 |
|2013|23.274199657409014 |
|2012|23.486330524955815 |
|2011|22.831230316985824 |
|2010|23.347699990556894 |
|2009|23.32151257796619  |
|2008|22.7818453225625   |
|2007|22.731810829963223 |
|2006|22.862693274978902 |
|2005|23.87910895028759  |
|2004|23.840810492739415 |
|2003|23.819686942500073 |
|2002|22.577967128952942 |
|2001|23.280010894145256 |
|2000|23.06669418194237  |
+----+-------------------+



A tabela mostra a variação anual da temperatura média ao longo de vários anos. É possível observar a flutuação ao longo dos anos, embora, com uma tendência geral de ligeiro aumento ao longo do tempo. As temperaturas variam entre 22.6ºC e 24ºC, onde o valor mais alto se registou no ano de 2021.

#### Média da Temperatura por Região

In [18]:
avg_temp_by_region = df_all_stations.groupBy("Region").agg(
    avg("AverageTemperatureC").alias("Average Temperature")
).orderBy(col("Average Temperature").desc())  

avg_temp_by_region.show()

+------+-------------------+
|Region|Average Temperature|
+------+-------------------+
|     N| 26.434321642212165|
|    NE| 25.931192444322907|
|    CO| 24.162870716936904|
|    SE| 22.081913355999855|
|     S|  19.04219842676723|
+------+-------------------+



Tendo em conta a tabela acima, e possível perceber que a região norte regista a maior média de temperatura, com 26.4ºC, seguida pela região nordeste. As restantes regiões mostram valores progressivamente mais baixos, com as região sul aquela a apresentar uma temperatura média menor. Este padrão é consistente com as condições climáticas das regiões brasileiras, já que, onde o norte e o nordeste do país são geralmente mais quentes enquanto o sul tem temperaturas mais amenas. 

#### Precipitação Total e Média por Ano

In [49]:
precipitation_by_year = df_all_stations.groupBy(year("Date").alias("Year")).agg(
    sum("TotalHourlyPrecipitationMm").alias("Total Precipitation"),
    avg("TotalHourlyPrecipitationMm").alias("Average Precipitation")
).orderBy(col("Year").desc())

precipitation_by_year.show(n=precipitation_by_year.count(), truncate=False)

+----+-------------------+---------------------+
|Year|Total Precipitation|Average Precipitation|
+----+-------------------+---------------------+
|2021|153510.9999999698  |0.1877805803532833   |
|2020|512770.79999990336 |0.15449612954307257  |
|2019|606981.599999815   |0.14703559265492921  |
|2018|616526.7999998596  |0.15168121414484784  |
|2017|578109.9999998764  |0.15290707531459044  |
|2016|514686.99999990134 |0.14271450001522326  |
|2015|499501.39999989583 |0.1434458497024536   |
|2014|517977.5999998969  |0.14836094169159894  |
|2013|590161.9999998495  |0.1633151060979648   |
|2012|472417.9999998962  |0.13390923827988635  |
|2011|554112.1999999371  |0.16888181124114796  |
|2010|467859.59999990807 |0.14205790110832586  |
|2009|590505.5999998706  |0.17932468270760715  |
|2008|437473.59999997437 |0.15500468585296676  |
|2007|198974.7999999852  |0.13141014337406587  |
|2006|97320.20000000359  |0.17091830641592773  |
|2005|56923.60000000292  |0.15729139898149186  |
|2004|56229.00000000

A tabela demonstra que o ano de 2021 apresenta uma precipitação total bastante baixa em relação aos restantes anos,no entanto, isto só acontece pois apenas existem dados até abril desse mesmo ano, ainda assim, apresenta a média de precipitação mais alta, o que pode ser explicado pela falta de dados relativa aos outros meses. É possível verificar, também que, a média tende a ser mais alta em anos com menor participação total, sugerindo que anos mais secos tendem a ter eventos de chiva mais intensos por curtos períodos, enquanto anos mais chuvosos distribuem a sua precipitação de uma maneira mais uniforme.

#### Precipitação Total e Média por Região

In [17]:
precipitation_summary = df_all_stations.groupBy("Region").agg(
    sum("TotalHourlyPrecipitationMm").alias("Total Precipitation"),
    avg("TotalHourlyPrecipitationMm").alias("Average Precipitation")
).orderBy(col("Average Precipitation").desc())

precipitation_summary.show()

+------+-------------------+---------------------+
|Region|Total Precipitation|Average Precipitation|
+------+-------------------+---------------------+
|     N| 1325296.8000010434|  0.22598713372510046|
|     S| 1641529.2000010447|   0.1866027624676102|
|    CO| 1385569.8000002718|  0.15587833664652576|
|    SE| 1946731.8000017058|  0.14664762320166794|
|    NE| 1302438.8000004084|   0.1006954770028099|
+------+-------------------+---------------------+



A tabela acima revela que a região norte é a que tem uma maior média de precipitação, o que está de acordo com o seu clima mais chuvoso. A região nordeste apresenta a média mais baixa, refletindo condições mais secas, as restantes regiões mostram médias intermediárias.

#### Média da Pressão Atmosférica por Ano

In [50]:
pressure_by_year = df_all_stations.groupBy(year("Date").alias("Year")).agg(
    avg("HourlyStationLevelAtmosphericPressureMb").alias("Average Atmospheric Pressure")
).orderBy(col("Year").desc())

pressure_by_year.show(n=pressure_by_year.count(), truncate=False)

+----+----------------------------+
|Year|Average Atmospheric Pressure|
+----+----------------------------+
|2021|957.9672361657883           |
|2020|962.2817993316169           |
|2019|965.4528882468895           |
|2018|965.4625021957719           |
|2017|965.6291658654668           |
|2016|966.7706190921162           |
|2015|966.4137845762514           |
|2014|966.7708289438913           |
|2013|966.3643847477864           |
|2012|965.9900454860181           |
|2011|964.4923064004962           |
|2010|965.3119595511387           |
|2009|965.9698935085712           |
|2008|963.8455885484424           |
|2007|960.660547263798            |
|2006|962.0436207841346           |
|2005|969.1448279768658           |
|2004|974.2131075640814           |
|2003|972.3525620968851           |
|2002|960.4224170202139           |
|2001|969.5007790964502           |
|2000|972.788415120118            |
+----+----------------------------+



A tabela mostra que a média da pressão atmosférica não variou muito ao longo dos anos apresentando valores relativamente estáveis com ligeiras flutuações.

#### Média da Pressão Atmosférica por Região

In [22]:
avg_pressure_by_region = df_all_stations.groupBy("Region").agg(
    avg("HourlyStationLevelAtmosphericPressureMb").alias("Average Atmospheric Pressure")
).orderBy(col("Average Atmospheric Pressure").desc())

avg_pressure_by_region.show()

+------+----------------------------+
|Region|Average Atmospheric Pressure|
+------+----------------------------+
|     N|           992.9902360124394|
|    NE|           979.8653690397847|
|     S|           961.0452742540027|
|    CO|           954.4505846352632|
|    SE|           948.9438967238226|
+------+----------------------------+



A tabela destaca as diferenças na pressão atmosférica média entre as regiões do Brasil. A região norte apresenta uma maior média enquanto a sudeste apresenta a menor.

#### Média da Humidade Relativa por Ano


In [51]:
avg_humidity_by_year = df_all_stations.groupBy(year("Date").alias("Year")).agg(
    avg("HourlyRelativeHumidityPercentage").alias("Average Relative Humidity")
).orderBy(col("Year").desc())

avg_humidity_by_year.show(n=avg_humidity_by_year.count(), truncate=False)

+----+-------------------------+
|Year|Average Relative Humidity|
+----+-------------------------+
|2021|74.32178392224117        |
|2020|71.0817764330573         |
|2019|71.08771968498063        |
|2018|72.05045512227213        |
|2017|70.64339544640502        |
|2016|70.69502247387291        |
|2015|71.1598214901096         |
|2014|71.59834607631352        |
|2013|72.61772174317309        |
|2012|70.47650058944464        |
|2011|72.98580186616236        |
|2010|71.74670975025225        |
|2009|75.04993499731698        |
|2008|72.16493174953274        |
|2007|70.08709369144822        |
|2006|73.11832362714175        |
|2005|71.98524450191904        |
|2004|73.49974276196478        |
|2003|71.97749759846302        |
|2002|70.73321687047667        |
|2001|72.3245795025007         |
|2000|72.38346806330411        |
+----+-------------------------+



Os dados da tabela indicam uma tendência de ligeira elevação da humidade ao longo do tempo, com os valores a oscilar entre 70% e 75%.

#### Média da Humidade Relativa por Região


In [26]:
avg_humidity_by_region = df_all_stations.groupBy("Region").agg(
    avg("HourlyRelativeHumidityPercentage").alias("Average Relative Humidity")
).orderBy(col("Average Relative Humidity").desc())

avg_humidity_by_region.show()

+------+-------------------------+
|Region|Average Relative Humidity|
+------+-------------------------+
|     S|        76.83653013475856|
|     N|        76.28801859466118|
|    SE|        72.12860252378644|
|    CO|        68.86778774163862|
|    NE|        67.93432189368656|
+------+-------------------------+



A tabela mostra que as regiões sul e norte apresentam as maiores médias de humidade, com valores bastante próximos, enquanto a região nordeste apresenta a menor média. Isto sugere que as regiões sul e norte podem experienciar condições atmosféricas mais húmidas, contrastando com o clima mais seco do nordeste.

### Média da Direção do Vento por Ano


In [52]:
avg_wind_direction_by_year = df_all_stations.groupBy(year("Date").alias("Year")).agg(
    avg("HourlyWindDirectionRadiusDegrees").alias("Average Wind Direction")
).orderBy(col("Year").desc())

avg_wind_direction_by_year.show(n=avg_wind_direction_by_year.count(), truncate=False)

+----+----------------------+
|Year|Average Wind Direction|
+----+----------------------+
|2021|160.85627925069298    |
|2020|160.1912317851104     |
|2019|157.6866976234016     |
|2018|157.13915316110575    |
|2017|152.94833782225052    |
|2016|154.41770015056525    |
|2015|150.60321380981918    |
|2014|148.08203168187288    |
|2013|150.17701265206273    |
|2012|148.17899133676522    |
|2011|150.76241738581834    |
|2010|150.80207248159448    |
|2009|153.11048208880754    |
|2008|152.5540573817686     |
|2007|151.1217243194371     |
|2006|148.4630204637897     |
|2005|142.4745771610311     |
|2004|150.0503061132619     |
|2003|148.44363249622614    |
|2002|153.21273485685765    |
|2001|152.9347341663503     |
|2000|159.29187065922807    |
+----+----------------------+



Nesta tabela, é possível perceber que as médias anuais oscilam ligeiramente ao longo dos anos, indicando pequenas variações nos padrões de vento. Os valores mantêm-se na faixa de 148 a 161 graus.

#### Média da Direção do Vento por Região


In [27]:
avg_wind_direction_by_region = df_all_stations.groupBy("Region").agg(
    avg("HourlyWindDirectionRadiusDegrees").alias("Average Wind Direction")
).orderBy(col("Average Wind Direction").desc())

avg_wind_direction_by_region.show()

+------+----------------------+
|Region|Average Wind Direction|
+------+----------------------+
|    CO|    169.39805575337024|
|     S|    158.86019001200194|
|    SE|    155.02657904262142|
|     N|    152.28756433435944|
|    NE|    135.35653115652855|
+------+----------------------+



Na tabela é possível perceber que a região centro regista a maior média de direção do vento, aproximadamente 169.4º, sugerindo ventos que predominam de uma direção mais ao sul. Por outro lado, a região nordeste apresenta a menor média, cerca de 135.4º, indicando uma predominância de ventos de uma direção mais ao sudeste.

#### Média da Temperatura do Ponto de Orvalho por Ano


In [46]:
avg_dew_point_by_year = df_all_stations.groupBy(year("Date").alias("Year")).agg(
    avg("DewPointTemperatureC").alias("Average Dew Point Temperature")
).orderBy(col("Year").desc())

avg_dew_point_by_year.show(n=avg_dew_point_by_year.count(), truncate=False)

+----+-----------------------------+
|Year|Average Dew Point Temperature|
+----+-----------------------------+
|2021|18.504984819608772           |
|2020|16.874844651442135           |
|2019|17.372899065363185           |
|2018|17.207445809229103           |
|2017|16.998025387795447           |
|2016|17.152111934028493           |
|2015|17.55166548923658            |
|2014|17.284736378702256           |
|2013|17.271427563343313           |
|2012|16.868397716826884           |
|2011|16.919815090527194           |
|2010|17.08926366723235            |
|2009|17.997507395364813           |
|2008|16.627906212077022           |
|2007|16.020131215446543           |
|2006|17.100717778136886           |
|2005|17.840883506171817           |
|2004|18.185355760202782           |
|2003|17.76026828598892            |
|2002|16.224641698456196           |
|2001|17.422435336645584           |
|2000|17.16485255607426            |
+----+-----------------------------+



Na tabela acima que mostra a média anual da temperatura do ponto de orvalho ao longo dos anos é possível observar variações moderadas nos valores, com uma tendência de variação entre 16ºC e 18ºC.

#### Média da Temperatura do Ponto de Orvalho por Região


In [30]:
avg_dew_point_by_region = df_all_stations.groupBy("Region").agg(
    avg("DewPointTemperatureC").alias("Average Dew Point Temperature")
).orderBy(col("Average Dew Point Temperature").desc())

avg_dew_point_by_region.show()

+------+-----------------------------+
|Region|Average Dew Point Temperature|
+------+-----------------------------+
|     N|            21.27023805524776|
|    NE|           18.552767558714176|
|    CO|            17.05682287465467|
|    SE|           16.010340119451296|
|     S|           14.311685888171214|
+------+-----------------------------+



É possível concluir que a região norte possui a maior média de temperatura do ponto de orvalho, aproximadamente 21.3°C, o que nos  indica que esta região tem um ambiente mais húmido. Em contraste, a região sul tem a menor média, cerca de 14.3°C, o que sugere que o clima desta região é mais seco.

### Conclusão e Considerações Finais

- Humidade e Temperatura por Região - A região norte mostra maiores médias de temperatura e humidade, o que indica maior probabilidade de precipitação.
A região sul apresenta temperaturas e humidade mais baixas, o que contribui para um clima mais ameno e menos propenso a chuvas intensas.

- Pressão Atmosférica - Existem variações significativas na pressão atmosférica entre regiões, com valores mais altos no norte e mais baixos no sudeste, o que irá influenciar os padrões de vento e precipitação.

- Direção do Vento - As variações na direção do vento ao longo dos anos em cada região irá reflitir mudanças nos padrões climáticos de cada região, o que pode afetar a distribuição de massas de ar húmido e os padrões de precipitação.

Assim estas variáveis podem influenciar os nossos modelos da seguinte maneira:

- Humidade Relativa - No caso do modelo de forma a prever a temperatura, a humidade influencia a retenção de calor afetando, assim, as temperaturas mínimas. No caso do modelo de previsão de chuva, uma alta humidade pode indicar maior probabilidade de precipitação.

- Pressão Atmosférica - Para prever a temperatura, uma baixa pressão atmosférica geralmente coincide com temperaturas mais elevadas. Para prever se chove,  uma baixa pressão atmosférica está associada a condições instáveis e consequente precipitação.

- Direção do Vento - No caso da previsão da temperatura, a direção do vento pode influenciar, já que, ventos provenientes de áreas quentes ou frias podem influenciar as temperaturas. Na previsão de chuva, o vento pode trazer ar húmido que pode aumentar a probabilidade de chuva.